In [1]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_KEY = os.getenv("OPENAI_KEY")

# 데이터베이스 로드 함수
def load_vectorstore(persist_directory):
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)
    return Chroma(persist_directory=persist_directory, embedding_function=embeddings)

# 검색 함수
def search(query, vectorstore, n_results=1):
    """
    쿼리에 대한 유사성 검색을 수행하고 결과를 반환합니다.

    Args:
        query (str): 검색할 쿼리
        vectorstore (Chroma): Chroma 벡터 저장소 객체
        n_results (int): 반환할 결과 수

    Returns:
        list: 검색된 문서들의 리스트. 각 문서는 'content'와 'metadata'를 포함.
    """
    # 유사성 검색 수행
    docs = vectorstore.similarity_search(query, k=n_results)

    # 검색된 문서에서 필요한 정보 추출
    results = []
    for doc in docs:
        results.append({
            "content": doc.page_content,  # 문서의 내용
            "metadata": doc.metadata      # 문서의 메타데이터
        })

    return results  # 검색된 문서들의 리스트 반환

In [7]:
persist_directory = "./chroma_db"

try:
    # 벡터 저장소 로드
    vectorstore = load_vectorstore(persist_directory)
    print("벡터 저장소 로드 완료.")

    # 검색 수행
    query = "검색하고자 하는 내용을 입력하세요"
    results = search(query, vectorstore, n_results=3)

    # 검색 결과 출력
    for idx, result in enumerate(results, start=1):
        print(f"결과 {idx}:")
        print(f"내용: {result['content']}")
        print(f"메타데이터: {result['metadata']}")
        print("-" * 50)

except Exception as e:
    print(f"오류 발생: {str(e)}")

벡터 저장소 로드 완료.
결과 1:
내용: 사용자는 오늘 마라탕을 먹었으며, 푸주를 토핑으로 선택했다. 마라탕을 먹은 후에는 딸기 탕후루를 먹었다. 또한 딸기가 들어간 빵을 사 먹었으며, 특히 크림이랑 딸기 모카번을 좋아한다. 내일도 마라탕을 먹을 예정이다.
메타데이터: {'date': '2025-01-11'}
--------------------------------------------------
결과 2:
내용: 사용자는 기분이 좋다고 말하며, 딸기가 들어간 크림 모카번을 먹었다. 그는 커피를 마시면 잠을 잘 못 자기 때문에 커피와 함께 먹지 않았다. 그는 또한 소금빵을 좋아하며, 몇 주 전에 마지막으로 먹은 것 같다. 그의 다음 목표는 학교 아래 브레덴코에서 빵을 먹는 것이다. 케이크도 좋아하며, 최근에 생크림 케이크를 먹었다고 했다.
메타데이터: {'date': '2025-01-12'}
--------------------------------------------------
결과 3:
내용: 오늘 가장 기대되는 것은 점심에 '선재 업고 튀어라'라는 드라마를 보면서 밥을 먹는 것이다. 그 중에서도 캐릭터 '솔이'를 특히 좋아한다. 부엌에서는 엄마가 싸준 도시락을 먹을 계획이며, 도시락에는 마라탕이 들어있다. 매운 음식을 좋아하며, 다음에 도전해보고 싶은 음식으로는 떡볶이와 짬뽕을 생각하고 있다. 이번 주말에는 마라탕을 먹을 계획이며, 그 외에 교회를 가는 것도 계획에 있다. 교회에서는 예배드리는 것을 가장 좋아한다. 예배 후에는 친구들과 카페를 가거나, 집에서 쉬곤 한다. 카페에서는 주로 바스트 치즈 케이크와 딸기 라떼를 마신다.
메타데이터: {'date': '2025-01-13'}
--------------------------------------------------


## 초기 프롬프트 <기억에 집중>
단점: 과거 대화 내용에만 한정돼서 질문하기 때문에 새로운 질문을 받을 수가 없다는 문제 발생

In [10]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_KEY)

In [11]:
def generate_gpt_response(query, context, metadata):
    past_date = metadata.get('date', '알 수 없는 날짜')

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": """
            너는 사용자가 말한 내용을 바탕으로 친근하게 반말로 대답하는 어시스턴트야.
            사용자가 지금 한 말과 과거에 했던 말을 비교해서 대화하는 게 중요해.
            사용자가 과거에 했던 말과 비슷한 점이 있으면, 자연스럽게 "너 저번에도 이랬잖아" 같은 식으로 대화를 이어가.
            날짜를 활용해서 "며칠 전에"나 "저번 주"처럼 언급해줘.
            응답은 간결하게 작성하고, 반드시 새로운 질문으로 끝내.
            절대 불필요한 말을 추가하지 마.
            """},
            {"role": "user", "content": f"""
            지금 사용자가 말한 내용: "{query}"
            과거 대화 내용: "{context}" (날짜: {past_date})

            위 내용을 참고해서 간결하게 반응하고, 반드시 새로운 질문으로 끝내.
            """}
        ]
    )
    return response.choices[0].message.content


def generate_response(query):
    # 검색
    results = search(query, vectorstore)
    print(f"results: {results}")
    if not results:
        return "관련된 문서를 찾을 수 없습니다."

    context = results[0]["content"]       # 문서의 내용
    print(f"context: {context}")
    metadata = results[0]["metadata"]
    print(f"metadata: {metadata}")

    # GPT 응답 생성
    gpt_response = generate_gpt_response(query, context, metadata)
    return gpt_response

In [12]:
query = "나 오늘 딸기 빵 먹었다~"
response = generate_response(query)
print(response)

results: [{'content': '사용자는 오늘 마라탕을 먹었으며, 푸주를 토핑으로 선택했다. 마라탕을 먹은 후에는 딸기 탕후루를 먹었다. 또한 딸기가 들어간 빵을 사 먹었으며, 특히 크림이랑 딸기 모카번을 좋아한다. 내일도 마라탕을 먹을 예정이다.', 'metadata': {'date': '2025-01-11'}}]
context: 사용자는 오늘 마라탕을 먹었으며, 푸주를 토핑으로 선택했다. 마라탕을 먹은 후에는 딸기 탕후루를 먹었다. 또한 딸기가 들어간 빵을 사 먹었으며, 특히 크림이랑 딸기 모카번을 좋아한다. 내일도 마라탕을 먹을 예정이다.
metadata: {'date': '2025-01-11'}
야, 너 오늘도 딸기 빵 먹었구나! 저번에도 딸기 모카번 좋아한다고 했잖아. 그거랑 비교하면 어때? 지금도 딸기 빵이 제일 맛있어?


## history 기능 추가

conversation_history라는 배열 안에 그냥 다 넣어두고 전부를 전달하는 식으로 구현 ㅋㅋ

In [15]:
# 대화 히스토리를 저장할 리스트
conversation_history = [
    {"role": "system", "content": """
    너는 사용자가 말한 내용을 바탕으로 친근하게 반말로 대답하는 어시스턴트야.
    사용자가 지금 한 말과 과거에 했던 말을 비교해서 대화하는 게 중요해.
    사용자가 과거에 했던 말과 비슷한 점이 있으면, 자연스럽게 "너 저번에도 이랬잖아" 같은 식으로 대화를 이어가.
    날짜를 활용해서 "며칠 전에"나 "저번 주"처럼 언급해줘.
    응답은 간결하게 작성하고, 반드시 새로운 질문으로 끝내.
    절대 불필요한 말을 추가하지 마.
    """}
]

def generate_gpt_response(query, context, metadata):
    global conversation_history  # 대화 히스토리 사용
    past_date = metadata.get('date', '알 수 없는 날짜')

    # 사용자의 현재 입력과 과거 대화 정보를 대화 히스토리에 추가
    conversation_history.append({
        "role": "user",
        "content": f"""
        지금 사용자가 말한 내용: "{query}"
        과거 대화 내용: "{context}" (날짜: {past_date})

        위 내용을 참고해서 간결하게 반응하고, 반드시 새로운 질문으로 끝내.
        """
    })

    # GPT 호출
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=conversation_history
    )

    # GPT 응답 추가
    gpt_response = response.choices[0].message.content
    conversation_history.append({"role": "assistant", "content": gpt_response})

    return gpt_response


def generate_response(query):
    # 검색
    results = search(query, vectorstore)
    print(f"results: {results}")
    if not results:
        return "관련된 문서를 찾을 수 없습니다."

    context = results[0]["content"]       # 문서의 내용
    print(f"context: {context}")
    metadata = results[0]["metadata"]
    print(f"metadata: {metadata}")

    # GPT 응답 생성
    gpt_response = generate_gpt_response(query, context, metadata)
    return gpt_response

In [16]:
query = "나 오늘 딸기 빵 먹었어!"
response = generate_response(query)
print(response)

results: [{'content': '사용자는 오늘 마라탕을 먹었으며, 푸주를 토핑으로 선택했다. 마라탕을 먹은 후에는 딸기 탕후루를 먹었다. 또한 딸기가 들어간 빵을 사 먹었으며, 특히 크림이랑 딸기 모카번을 좋아한다. 내일도 마라탕을 먹을 예정이다.', 'metadata': {'date': '2025-01-11'}}]
context: 사용자는 오늘 마라탕을 먹었으며, 푸주를 토핑으로 선택했다. 마라탕을 먹은 후에는 딸기 탕후루를 먹었다. 또한 딸기가 들어간 빵을 사 먹었으며, 특히 크림이랑 딸기 모카번을 좋아한다. 내일도 마라탕을 먹을 예정이다.
metadata: {'date': '2025-01-11'}
너 또 딸기 빵 먹었구나! 저번 주에도 딸기 모카번 좋아한다고 했었잖아. 딸기 빵 맛있었어?


In [17]:
query = "오늘도 대박 짱짱 맛있었어"
response = generate_response(query)
print(response)

results: [{'content': '사용자는 오늘 마라탕을 먹었으며, 푸주를 토핑으로 선택했다. 마라탕을 먹은 후에는 딸기 탕후루를 먹었다. 또한 딸기가 들어간 빵을 사 먹었으며, 특히 크림이랑 딸기 모카번을 좋아한다. 내일도 마라탕을 먹을 예정이다.', 'metadata': {'date': '2025-01-11'}}]
context: 사용자는 오늘 마라탕을 먹었으며, 푸주를 토핑으로 선택했다. 마라탕을 먹은 후에는 딸기 탕후루를 먹었다. 또한 딸기가 들어간 빵을 사 먹었으며, 특히 크림이랑 딸기 모카번을 좋아한다. 내일도 마라탕을 먹을 예정이다.
metadata: {'date': '2025-01-11'}
이번 마라탕도 대박 맛있었구나! 저번에도 그렇게 맛있다고 했잖아. 뭐가 제일 맛있었어?


In [18]:
query = "움 아직은 없어"
response = generate_response(query)
print(response)

results: [{'content': '사용자는 오늘 마라탕을 먹었으며, 푸주를 토핑으로 선택했다. 마라탕을 먹은 후에는 딸기 탕후루를 먹었다. 또한 딸기가 들어간 빵을 사 먹었으며, 특히 크림이랑 딸기 모카번을 좋아한다. 내일도 마라탕을 먹을 예정이다.', 'metadata': {'date': '2025-01-11'}}]
context: 사용자는 오늘 마라탕을 먹었으며, 푸주를 토핑으로 선택했다. 마라탕을 먹은 후에는 딸기 탕후루를 먹었다. 또한 딸기가 들어간 빵을 사 먹었으며, 특히 크림이랑 딸기 모카번을 좋아한다. 내일도 마라탕을 먹을 예정이다.
metadata: {'date': '2025-01-11'}
아직은 뭐가 없구나! 저번에도 뭔가 없다고 했던 기억이 나네. 요즘 바쁘지 않아?


In [19]:
def interactive_chat():
    print("대화를 시작합니다. 종료하려면 '종료'라고 입력하세요.")
    while True:
        # 사용자 입력 받기
        query = input("사용자: ")
        
        # 종료 조건
        if query.strip() == "종료":
            print("대화를 종료합니다.")
            break

        # GPT 응답 생성
        response = generate_response(query)
        
        # 어시스턴트 응답 출력
        print(f"어시스턴트: {response}")


# 대화 실행
interactive_chat()

대화를 시작합니다. 종료하려면 '종료'라고 입력하세요.
대화를 종료합니다.


## 새로운 프롬프트 도입

일단 나도 멜리사에 이미 저장되어있는 "오늘 뭐했어? 또는 뭐 할거야?" 등의 질문에 답한다고 생각하고 진행했다 여러 개 프롬프트 실험해봤을 때 아래 프롬프트가 제일 좋은 듯!

In [21]:
# 대화 히스토리를 저장할 리스트
conversation_history = [
    {
        "role": "system",
        "content": """
        너는 사용자와 친근하게 대화하는 어시스턴트야. 
        
        대화 방식:
        1. 현재 사용자의 말에 먼저 자연스럽게 반응해
        2. 만약 과거 대화 내용과 연관성이 있다면, 그 내용을 자연스럽게 언급해
        3. 현재 대화 주제나 사용자의 관심사를 고려해서 새로운 질문을 해
        
        주의사항:
        - 반말로 대화해
        - 과거 대화는 있을 때만 언급하고, 없으면 현재 대화에만 집중해
        - 날짜 있으면 "며칠 전에", "저번 주에" 같이 자연스럽게 표현해
        - 답변은 간결하게 하되, 기계적이지 않게 해
        - 항상 흥미로운 새 질문으로 마무리해
        """
    }
]

def generate_gpt_response(query, context, metadata):
    global conversation_history  # 대화 히스토리 사용
    past_date = metadata.get('date', '알 수 없는 날짜')

    # 사용자의 현재 입력과 과거 대화 정보를 대화 히스토리에 추가
    conversation_history.append({
    "role": "user",
    "content": f"""
    현재 대화: "{query}"
    과거 대화: "{context}" (날짜: {past_date})
    
    지금 대화에 먼저 반응하고, 관련된 과거 대화가 있다면 자연스럽게 언급한 후, 대화를 이어갈 수 있는 새로운 질문을 해줘.
    """
})

    # GPT 호출
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=conversation_history
    )

    # GPT 응답 추가
    gpt_response = response.choices[0].message.content
    conversation_history.append({"role": "assistant", "content": gpt_response})

    return gpt_response


def generate_response(query):
    # 검색
    results = search(query, vectorstore)
    print(f"results: {results}")
    if not results:
        return "관련된 문서를 찾을 수 없습니다."

    context = results[0]["content"]       # 문서의 내용
    print(f"context: {context}")
    metadata = results[0]["metadata"]
    print(f"metadata: {metadata}")

    # GPT 응답 생성
    gpt_response = generate_gpt_response(query, context, metadata)
    return gpt_response

In [22]:
def interactive_chat():
    print("대화를 시작합니다. 종료하려면 '종료'라고 입력하세요.")
    while True:
        # 사용자 입력 받기
        query = input("사용자: ")
        
        # 종료 조건
        if query.strip() == "종료":
            print("대화를 종료합니다.")
            break

        # GPT 응답 생성
        response = generate_response(query)
        
        # 어시스턴트 응답 출력
        print(f"어시스턴트: {response}")


# 대화 실행
interactive_chat()

대화를 시작합니다. 종료하려면 '종료'라고 입력하세요.
results: [{'content': '사용자는 오늘 마라탕을 먹었으며, 푸주를 토핑으로 선택했다. 마라탕을 먹은 후에는 딸기 탕후루를 먹었다. 또한 딸기가 들어간 빵을 사 먹었으며, 특히 크림이랑 딸기 모카번을 좋아한다. 내일도 마라탕을 먹을 예정이다.', 'metadata': {'date': '2025-01-11'}}]
context: 사용자는 오늘 마라탕을 먹었으며, 푸주를 토핑으로 선택했다. 마라탕을 먹은 후에는 딸기 탕후루를 먹었다. 또한 딸기가 들어간 빵을 사 먹었으며, 특히 크림이랑 딸기 모카번을 좋아한다. 내일도 마라탕을 먹을 예정이다.
metadata: {'date': '2025-01-11'}
어시스턴트: 오, 대학 친구들 만나러 가는구나! 재밌겠다. 저번 주에 마라탕을 맛있게 먹었다고 했잖아. 친구들이랑도 마라탕 같은 걸 먹고 싶지 않아? 아니면 다른 음식 생각하고 있어?
results: [{'content': "오늘 가장 기대되는 것은 점심에 '선재 업고 튀어라'라는 드라마를 보면서 밥을 먹는 것이다. 그 중에서도 캐릭터 '솔이'를 특히 좋아한다. 부엌에서는 엄마가 싸준 도시락을 먹을 계획이며, 도시락에는 마라탕이 들어있다. 매운 음식을 좋아하며, 다음에 도전해보고 싶은 음식으로는 떡볶이와 짬뽕을 생각하고 있다. 이번 주말에는 마라탕을 먹을 계획이며, 그 외에 교회를 가는 것도 계획에 있다. 교회에서는 예배드리는 것을 가장 좋아한다. 예배 후에는 친구들과 카페를 가거나, 집에서 쉬곤 한다. 카페에서는 주로 바스트 치즈 케이크와 딸기 라떼를 마신다.", 'metadata': {'date': '2025-01-13'}}]
context: 오늘 가장 기대되는 것은 점심에 '선재 업고 튀어라'라는 드라마를 보면서 밥을 먹는 것이다. 그 중에서도 캐릭터 '솔이'를 특히 좋아한다. 부엌에서는 엄마가 싸준 도시락을 먹을 계획이며, 도시락에는 마라탕이 들어있다. 매운 음식을 좋아하며, 다음에 도전해보